# Part 1 - Source Data Import (into SQLite3), Cleaning, and Transformation

## Importing of Libraries


In [3]:
# importing sql libraries
import sqlite3
from sqlalchemy import create_engine

#importing libraries required for data wrangling/transformation
import pandas as pd
from sklearn.impute import KNNImputer

## Reading in source data and importing into SQL database

In [4]:
#reading in source data (.txv)
raw_data = pd.read_csv('../data/nhgh.tsv', sep='\t', engine='python')

#instantiation of SQLite3 database
sqliteConnection = sqlite3.connect('../data/nhgh.db')

#importing data into database
cursor = sqliteConnection.cursor()
engine = create_engine('sqlite:///..\\data\\nhgh.db',
                       echo=False)
raw_data.to_sql("nhgh", con=engine, index=False, if_exists='replace')

sqliteConnection.close()

## Data Transformation and Cleaning 

### General Data Analysis

In [5]:
#Connecting to nhgh database
sqliteConnection = sqlite3.connect('../data/nhgh.db')

#Querying from nhgh database
data = pd.read_sql('SELECT * FROM nhgh', sqliteConnection)
data

,seqn,sex,age,re,income,tx,dx,wt,ht,bmi,leg,arml,armc,waist,tri,sub,gh,albumin,bun,SCr
0,51624,male,34.166667,Non-Hispanic White,"[25000,35000)",0,0,87.4,164.7,32.22,41.5,40.0,36.4,100.4,16.4,24.9,5.2,4.8,6.0,0.94
1,51626,male,16.833333,Non-Hispanic Black,"[45000,55000)",0,0,72.3,181.3,22.00,42.0,39.5,26.6,74.7,10.2,10.5,5.7,4.6,9.0,0.89
2,51628,female,60.166667,Non-Hispanic Black,"[10000,15000)",1,1,116.8,166.0,42.39,35.3,39.0,42.2,118.2,29.6,35.6,6.0,3.9,10.0,1.11
3,51629,male,26.083333,Mexican American,"[25000,35000)",0,0,97.6,173.0,32.61,41.7,38.7,37.0,103.7,19.0,23.2,5.1,4.2,8.0,0.80
4,51630,female,49.666667,Non-Hispanic White,"[35000,45000)",0,0,86.7,168.4,30.57,37.5,36.1,33.3,107.8,30.3,28.0,5.3,4.3,13.0,0.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6790,62155,male,33.000000,Mexican American,"[35000,45000)",0,0,94.3,163.5,35.28,34.4,34.7,35.5,112.3,20.2,NaN,5.4,4.1,10.0,0.97
6791,62156,female,48.916667,Non-Hispanic White,"[0,5000)",0,1,87.1,156.9,35.38,33.9,34.5,37.0,99.4,28.6,25.4,5.5,4.1,7.0,0.89
6792,62157,male,27.500000,Other Hispanic,"[35000,45000)",0,0,57.0,164.3,21.12,35.3,33.7,29.6,73.2,4.2,6.8,5.6,4.5,11.0,0.94
6793,62158,male,75.750000,Non-Hispanic Black,"[10000,15000)",0,0,75.1,162.7,28.37,38.6,36.8,31.2,104.0,19.8,21.1,5.4,4.0,19.0,1.34


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6795 entries, 0 to 6794
Data columns (total 20 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   seqn     6795 non-null   int64  
 1   sex      6795 non-null   object 
 2   age      6795 non-null   float64
 3   re       6795 non-null   object 
 4   income   6475 non-null   object 
 5   tx       6795 non-null   int64  
 6   dx       6795 non-null   int64  
 7   wt       6795 non-null   float64
 8   ht       6795 non-null   float64
 9   bmi      6795 non-null   float64
 10  leg      6564 non-null   float64
 11  arml     6616 non-null   float64
 12  armc     6607 non-null   float64
 13  waist    6556 non-null   float64
 14  tri      6314 non-null   float64
 15  sub      5824 non-null   float64
 16  gh       6795 non-null   float64
 17  albumin  6706 non-null   float64
 18  bun      6706 non-null   float64
 19  SCr      6706 non-null   float64
dtypes: float64(14), int64(3), object(3)
memory usage: 1.

Number of columns aligns with the data dictionary (https://hbiostat.org/data/repo/nhgh)

In [7]:
data.describe()

,seqn,age,tx,dx,wt,ht,bmi,leg,arml,armc,waist,tri,sub,gh,albumin,bun,SCr
count,6795.000000,6795.000000,6795.000000,6795.000000,6795.000000,6795.000000,6795.000000,6564.000000,6616.000000,6607.000000,6556.000000,6314.000000,5824.000000,6795.000000,6706.000000,6706.000000,6706.000000
mean,56872.080500,44.285700,0.091832,0.134511,79.370625,167.042958,28.321741,38.409324,36.874607,32.485152,96.254149,18.787726,19.961556,5.676586,4.273621,12.917686,0.878627
std,3043.535776,20.594593,0.288810,0.341225,21.930903,10.264984,6.950110,3.876902,2.781616,5.297660,17.059193,8.319393,8.369083,0.964700,0.326545,5.717571,0.445238
min,51624.000000,12.000000,0.000000,0.000000,28.000000,123.300000,13.180000,20.400000,24.800000,16.800000,52.000000,2.600000,3.800000,4.000000,2.500000,1.000000,0.140000
25%,54245.500000,25.666667,0.000000,0.000000,64.000000,159.600000,23.430000,36.000000,35.000000,28.850000,83.500000,12.000000,13.000000,5.200000,4.100000,9.000000,0.700000
50%,56873.000000,43.750000,0.000000,0.000000,76.300000,166.600000,27.290000,38.400000,36.800000,32.100000,95.300000,17.900000,19.400000,5.500000,4.300000,12.000000,0.830000
75%,59510.500000,61.333333,0.000000,0.000000,91.100000,174.500000,31.880000,41.000000,38.800000,35.600000,106.900000,25.000000,26.200000,5.800000,4.500000,15.000000,0.980000
max,62160.000000,80.000000,1.000000,1.000000,239.400000,202.700000,84.870000,50.600000,47.000000,61.000000,179.000000,41.100000,40.400000,16.400000,5.300000,90.000000,15.660000


In [8]:
#Checking the data type of the values under [income]
type(data.loc[1,'income'])

str

In [9]:
#BMI max seems a little high. checking the top few ones to see if the data is correct
data.sort_values('bmi', ascending=False).head()

,seqn,sex,age,re,income,tx,dx,wt,ht,bmi,leg,arml,armc,waist,tri,sub,gh,albumin,bun,SCr
6142,61132,female,46.166667,Non-Hispanic Black,"[55000,65000)",0,0,196.6,152.2,84.87,NaN,36.2,54.1,152.5,NaN,NaN,5.8,4.0,18.0,1.23
480,52315,female,52.500000,Non-Hispanic Black,"[45000,55000)",1,1,230.7,168.5,81.25,NaN,NaN,NaN,NaN,NaN,NaN,7.0,3.3,30.0,1.66
2662,55757,female,30.250000,Other Hispanic,"[65000,75000)",0,0,195.3,165.5,71.30,NaN,NaN,NaN,NaN,NaN,NaN,5.2,3.8,14.0,0.72
6502,61693,female,34.000000,Other Hispanic,"[15000,20000)",1,1,188.9,165.9,68.63,NaN,41.5,52.5,165.0,NaN,NaN,7.2,3.9,10.0,0.95
5210,59688,female,39.416667,Non-Hispanic White,"[25000,35000)",1,1,170.2,158.4,67.83,37.6,NaN,58.5,163.5,NaN,NaN,7.2,3.5,14.0,0.84


In [10]:
#using row index = 6142 as an example,
#Calculate BMI = wt (kg) / (ht(m) * ht(m))
196.6 / (1.522*1.522)

84.87000126053104

<h4>Summary of Findings</h4>

<b>Data Types</b>

From the info() and describe() methods, I know that values in [seqn], [tx], and [dx] are integers where the latter two are binary. The remaining floats are continuous variables. [sex] and [re] are objects, as expected, but [income] is a string. It is better to convert [income] to a continuous variable and [sex] to a binary variable before performing any regressions.

<b>Sensemake the data</b>

Using the .describe() method, I can check for any outliers based on the min and max values of any continuous feature. From this, I see that the max value for [bmi] is quite far from normal. A quick check on that specific patient's data was done and concluded that the calculation of the BMI using the [wt] and [ht] matches the value in [bmi]. Thus, I can be rest assured that it is not an outlier.

More domain knowledge is of course beneficial in this step to quickly identify any values that are grossly out of range. 

<b>Target Variable: dx vs. gh</b>

For this assessment, I will be using [dx] (Diagnosed with DM or Pre-DM) as the target variable instead of [gh] (Glycohemoglobin % Index) as a high Glycohemoglobin % Index does not directly indicate that the patient is diabetic (it may be due to certain medications/steroids that causes the index to be higher).(source:https://www.goodrx.com/conditions/diabetes/normal-a1c#)

As such, [dx] will be used as the target variable.






### Conversion of Data Types


<h4>Converting [income] from categorical bands to a continuous variable</h4>

As mentioned, I will be converting [income] to a continuous variable. This will be done by replacing the band with a single number that is a mean between the two limits. For those categories that aren't bounded by two values, I will be inputing the number itself if it is the lower bound. for "< 20000", which is an upper bound, 10000 will be inputed as it is the mean of 0 and 20000.

In [11]:
#Checking the different income categories
data['income'][data['albumin'] > 0].value_counts()

income
>= 100000         868
[25000,35000)     833
[35000,45000)     605
[75000,100000)    559
[20000,25000)     555
[10000,15000)     524
[45000,55000)     512
[15000,20000)     452
[55000,65000)     366
[5000,10000)      311
[65000,75000)     270
[0,5000)          234
> 20000           231
< 20000            73
Name: count, dtype: int64

In [12]:
#extracting the values in income column to replace with the mean between bands.
#Note: >20000 is too large a band so I will impute it another way.
a = data['income'].value_counts().keys().to_list()
b = [
    100000, 
    30000,
    40000,
    87500,
    22500,
    12500,
    50000,
    17500,
    60000,
    7500,
    70000,
    2500,
    20000,
    10000   
]

data = data.replace(a , b)
data

,seqn,sex,age,re,income,tx,dx,wt,ht,bmi,leg,arml,armc,waist,tri,sub,gh,albumin,bun,SCr
0,51624,male,34.166667,Non-Hispanic White,30000.0,0,0,87.4,164.7,32.22,41.5,40.0,36.4,100.4,16.4,24.9,5.2,4.8,6.0,0.94
1,51626,male,16.833333,Non-Hispanic Black,50000.0,0,0,72.3,181.3,22.00,42.0,39.5,26.6,74.7,10.2,10.5,5.7,4.6,9.0,0.89
2,51628,female,60.166667,Non-Hispanic Black,12500.0,1,1,116.8,166.0,42.39,35.3,39.0,42.2,118.2,29.6,35.6,6.0,3.9,10.0,1.11
3,51629,male,26.083333,Mexican American,30000.0,0,0,97.6,173.0,32.61,41.7,38.7,37.0,103.7,19.0,23.2,5.1,4.2,8.0,0.80
4,51630,female,49.666667,Non-Hispanic White,40000.0,0,0,86.7,168.4,30.57,37.5,36.1,33.3,107.8,30.3,28.0,5.3,4.3,13.0,0.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6790,62155,male,33.000000,Mexican American,40000.0,0,0,94.3,163.5,35.28,34.4,34.7,35.5,112.3,20.2,NaN,5.4,4.1,10.0,0.97
6791,62156,female,48.916667,Non-Hispanic White,2500.0,0,1,87.1,156.9,35.38,33.9,34.5,37.0,99.4,28.6,25.4,5.5,4.1,7.0,0.89
6792,62157,male,27.500000,Other Hispanic,40000.0,0,0,57.0,164.3,21.12,35.3,33.7,29.6,73.2,4.2,6.8,5.6,4.5,11.0,0.94
6793,62158,male,75.750000,Non-Hispanic Black,12500.0,0,0,75.1,162.7,28.37,38.6,36.8,31.2,104.0,19.8,21.1,5.4,4.0,19.0,1.34


In [13]:
#check the data type
type(data.loc[1,'income'])

numpy.float64

<h4>Converting [sex] from categorical bands to a binary variable</h4>

As mentioned, I will be converting [sex] to a binary variable. This will be done by changing the feature name to be [sex_male] and have the values 1 or 0 representing True or False respectively.

In [14]:
data['sex'] = data['sex'].replace(['male','female'],[1,0])
data.rename(columns={'sex':'sex_male'}, inplace=True)

data

,seqn,sex_male,age,re,income,tx,dx,wt,ht,bmi,leg,arml,armc,waist,tri,sub,gh,albumin,bun,SCr
0,51624,1,34.166667,Non-Hispanic White,30000.0,0,0,87.4,164.7,32.22,41.5,40.0,36.4,100.4,16.4,24.9,5.2,4.8,6.0,0.94
1,51626,1,16.833333,Non-Hispanic Black,50000.0,0,0,72.3,181.3,22.00,42.0,39.5,26.6,74.7,10.2,10.5,5.7,4.6,9.0,0.89
2,51628,0,60.166667,Non-Hispanic Black,12500.0,1,1,116.8,166.0,42.39,35.3,39.0,42.2,118.2,29.6,35.6,6.0,3.9,10.0,1.11
3,51629,1,26.083333,Mexican American,30000.0,0,0,97.6,173.0,32.61,41.7,38.7,37.0,103.7,19.0,23.2,5.1,4.2,8.0,0.80
4,51630,0,49.666667,Non-Hispanic White,40000.0,0,0,86.7,168.4,30.57,37.5,36.1,33.3,107.8,30.3,28.0,5.3,4.3,13.0,0.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6790,62155,1,33.000000,Mexican American,40000.0,0,0,94.3,163.5,35.28,34.4,34.7,35.5,112.3,20.2,NaN,5.4,4.1,10.0,0.97
6791,62156,0,48.916667,Non-Hispanic White,2500.0,0,1,87.1,156.9,35.38,33.9,34.5,37.0,99.4,28.6,25.4,5.5,4.1,7.0,0.89
6792,62157,1,27.500000,Other Hispanic,40000.0,0,0,57.0,164.3,21.12,35.3,33.7,29.6,73.2,4.2,6.8,5.6,4.5,11.0,0.94
6793,62158,1,75.750000,Non-Hispanic Black,12500.0,0,0,75.1,162.7,28.37,38.6,36.8,31.2,104.0,19.8,21.1,5.4,4.0,19.0,1.34


### Analysis of Null Data

In [15]:
data.isnull().sum()

seqn          0
sex_male      0
age           0
re            0
income      320
tx            0
dx            0
wt            0
ht            0
bmi           0
leg         231
arml        179
armc        188
waist       239
tri         481
sub         971
gh            0
albumin      89
bun          89
SCr          89
dtype: int64

The null count for all columns corresponds to the information provided in the data dictionary. (https://hbiostat.org/data/repo/nhgh)


<b>Checking for biases in Null Data</b>

Checking if the proportion of null is overwhelmingly biased towards or against the target [dx]. This is to verify if a patient is prescribed extra tests/measurements if they are/aren't diabetic and may be a good predictor for our model.

In [16]:
print("\033[4mProportion of dx_1 population:\033[0m")
print(f"{round(data['seqn'][data['dx']==1].count()/data['seqn'].count(), 3)}")

Proportion of dx_1 population:
0.135


In [17]:
df_compare = pd.DataFrame()
df_compare['raw'] = data.isnull().sum()
df_compare['dx_0'] = data[data['dx']==0].isnull().sum()
df_compare['dx_1'] = data[data['dx']==1].isnull().sum()

df_compare['dx_0_perc'] = df_compare.apply(lambda x: round(x['dx_0']/x['raw'], 3) if x['dx_0'] > 0 else '-', axis = 1)
df_compare['dx_1_perc'] = df_compare.apply(lambda x: round(x['dx_1']/x['raw'], 3) if x['dx_1'] > 0 else '-', axis = 1)

df_compare

,raw,dx_0,dx_1,dx_0_perc,dx_1_perc
seqn,0,0,0,-,-
sex_male,0,0,0,-,-
age,0,0,0,-,-
re,0,0,0,-,-
income,320,296,24,0.925,0.075
tx,0,0,0,-,-
dx,0,0,0,-,-
wt,0,0,0,-,-
ht,0,0,0,-,-
bmi,0,0,0,-,-


From this, although there is some marginal bias towards people with diabetes having null values, it isn't overwhelming and as such, not notable.


### Filling in Null Data

<h4>Missing Income Data</h4>

Missing income data will be filled with median income by race using the following source:
<b>Household Income in the Nashville Area, Tennessee</b>

- Mexican American: \$33071                    
- Other Hispanic: \$37146                    
- Non-Hispanic White: \$50273                  
- Non-Hispanic Black: \$33293                  
- Other Race Including Multi-Racial: \$35973   

(Source: https://statisticalatlas.com/metro-area/Tennessee/Nashville/Household-Income#figure/median-household-income-by-race)

In [18]:
print('\033[4mBefore\033[0m')
data['re'][data['income'].isnull()].value_counts()


Before


re
Mexican American                     95
Other Hispanic                       81
Non-Hispanic White                   73
Non-Hispanic Black                   49
Other Race Including Multi-Racial    22
Name: count, dtype: int64

In [19]:
income_dict = {
    'Mexican American': 33071,
    'Other Hispanic': 37146, 
    'Non-Hispanic White': 50273, 
    'Non-Hispanic Black': 33293, 
    'Other Race Including Multi-Racial': 35973
    }

data['income'] = data['income'].fillna(data['re'].map(income_dict))

In [20]:
print('\033[4mAfter\033[0m')
data['re'][data['income'].isnull()].value_counts()

After


Series([], Name: count, dtype: int64)

<h4>Other Missing Data</h4>

For other missing data, I will be using KNN Imputer to determine the mean value based on the Nth closest neighbours. Prior to executing this, unique identifiers like [seqn], target column [dx], and non-numerical features [re], will be removed. KNNImputer is used instead of imputing from any literature review because specific clinical data/measurements is unique to each individual. It is better to extrapolate/intrapolate the missing data from nearest neighbours (one of similar bmi for example).

NOTE: Imputing of values here is only used for EDA. Another round of KNNImputer fit.transform will be done prior to model building in order to prevent data leakage from the Train dataset to the Test dataset

In [21]:
data_knn = data.drop(columns=['seqn', 're', 'dx'])
data_knn.head()

,sex_male,age,income,tx,wt,ht,bmi,leg,arml,armc,waist,tri,sub,gh,albumin,bun,SCr
0,1,34.166667,30000.0,0,87.4,164.7,32.22,41.5,40.0,36.4,100.4,16.4,24.9,5.2,4.8,6.0,0.94
1,1,16.833333,50000.0,0,72.3,181.3,22.00,42.0,39.5,26.6,74.7,10.2,10.5,5.7,4.6,9.0,0.89
2,0,60.166667,12500.0,1,116.8,166.0,42.39,35.3,39.0,42.2,118.2,29.6,35.6,6.0,3.9,10.0,1.11
3,1,26.083333,30000.0,0,97.6,173.0,32.61,41.7,38.7,37.0,103.7,19.0,23.2,5.1,4.2,8.0,0.80
4,0,49.666667,40000.0,0,86.7,168.4,30.57,37.5,36.1,33.3,107.8,30.3,28.0,5.3,4.3,13.0,0.79


In [22]:
imputer = KNNImputer(n_neighbors=5)
data_nonull = pd.DataFrame(imputer.fit_transform(data_knn),columns = data_knn.columns)

In [23]:
#checking the result of the KNNimputer
data_nonull.isnull().sum()

sex_male    0
age         0
income      0
tx          0
wt          0
ht          0
bmi         0
leg         0
arml        0
armc        0
waist       0
tri         0
sub         0
gh          0
albumin     0
bun         0
SCr         0
dtype: int64

In [24]:
#adding back the previously dropped columns
data_nonull['seqn'] = data['seqn']
data_nonull['re'] = data['re']
data_nonull['dx'] = data['dx']
data_nonull = data_nonull.reindex(columns=data.columns.tolist()) #sorting it back to the original order
data_nonull

,seqn,sex_male,age,re,income,tx,dx,wt,ht,bmi,leg,arml,armc,waist,tri,sub,gh,albumin,bun,SCr
0,51624,1.0,34.166667,Non-Hispanic White,30000.0,0.0,0,87.4,164.7,32.22,41.5,40.0,36.4,100.4,16.4,24.9,5.2,4.8,6.0,0.94
1,51626,1.0,16.833333,Non-Hispanic Black,50000.0,0.0,0,72.3,181.3,22.00,42.0,39.5,26.6,74.7,10.2,10.5,5.7,4.6,9.0,0.89
2,51628,0.0,60.166667,Non-Hispanic Black,12500.0,1.0,1,116.8,166.0,42.39,35.3,39.0,42.2,118.2,29.6,35.6,6.0,3.9,10.0,1.11
3,51629,1.0,26.083333,Mexican American,30000.0,0.0,0,97.6,173.0,32.61,41.7,38.7,37.0,103.7,19.0,23.2,5.1,4.2,8.0,0.80
4,51630,0.0,49.666667,Non-Hispanic White,40000.0,0.0,0,86.7,168.4,30.57,37.5,36.1,33.3,107.8,30.3,28.0,5.3,4.3,13.0,0.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6790,62155,1.0,33.000000,Mexican American,40000.0,0.0,0,94.3,163.5,35.28,34.4,34.7,35.5,112.3,20.2,28.5,5.4,4.1,10.0,0.97
6791,62156,0.0,48.916667,Non-Hispanic White,2500.0,0.0,1,87.1,156.9,35.38,33.9,34.5,37.0,99.4,28.6,25.4,5.5,4.1,7.0,0.89
6792,62157,1.0,27.500000,Other Hispanic,40000.0,0.0,0,57.0,164.3,21.12,35.3,33.7,29.6,73.2,4.2,6.8,5.6,4.5,11.0,0.94
6793,62158,1.0,75.750000,Non-Hispanic Black,12500.0,0.0,0,75.1,162.7,28.37,38.6,36.8,31.2,104.0,19.8,21.1,5.4,4.0,19.0,1.34


### Export to SQL database

In [23]:
#instantiation of SQLite3 database
sqliteConnection = sqlite3.connect('../data/nhgh_nonull.db')

#importing data into database
cursor = sqliteConnection.cursor()
engine = create_engine('sqlite:///..\\data\\nhgh_nonull.db',
                       echo=False)
data_nonull.to_sql("nhgh_nonull", con=engine, index=False, if_exists='replace')

sqliteConnection.close()